In [1]:
import pandas as pd
import os
import dask.dataframe as dd
from dask.distributed import Client

pd.set_option("mode.copy_on_write", True)

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
import os

print(f"I have {os.cpu_count()} logical cores.")

I have 8 logical cores.


In [5]:
client = Client()

# Define the columns you want to keep
required_columns = [
    "BUYER_COUNTY",
    "BUYER_STATE",
    "TRANSACTION_DATE",
    "DOSAGE_UNIT",
    "CALC_BASE_WT_IN_GM",
    "MME",
]

large_df = dd.read_csv(
    "/Users/keonnartey/Downloads/arcos_all_washpost.tsv",
    sep="\t",
    usecols=required_columns,
    dtype={
        "MME": "float64",
        "CALC_BASE_WT_IN_GM": "float64",
    },  # Specify the dtype for the columns
)

# Extract year
large_df["Year"] = dd.to_datetime(large_df.TRANSACTION_DATE, format="%Y-%m-%d").dt.year

# Group by BUYER_COUNTY, BUYER_STATE, Year and sum MME and CALC_BASE_WT_IN_GM

grouped_df = (
    large_df.groupby(["BUYER_COUNTY", "BUYER_STATE", "Year"])["MME"].sum().reset_index()
)

/Users/keonnartey/miniconda3/envs/dask_env/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54839 instead
  warnings.warn(


In [6]:
result = grouped_df.compute()
result.head()

,BUYER_COUNTY,BUYER_STATE,Year,MME
0,ADAMS,IN,2006,3.290648e+06
1,ADAMS,IN,2007,3.786557e+06
2,ADAMS,IN,2008,3.946667e+06
3,ADAMS,IN,2009,4.369181e+06
4,ADAMS,IN,2010,5.176397e+06
